In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
#full names are what I'm using as the unique id 
#

In [3]:
df_hire=pd.read_csv('../Data/hiring_data.csv')
df_hire.head()

,Last Name,First Name,Hire Date,Termination Date,Years of Current Service,Tenure Category,Primary Termination Category,Primary Termination Reason,Secondary Termination Reasons,Involuntary Termination,...,Location,Company,Organization,Manager,HR Business Partner,Cost Center,Job Family Base,Job Family,Management Level,Job Classification
0,Tavelli,Matthew,1/18/2010,4/6/2018,8.22,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Contract Loss,NaN,NaN,...,"USA, MO, Kansas City (2300 Main St)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Darin Morris (528274),NaN,18 ARTS - Non Client Site,Technology,SD Project/Program Manager,Associate,2 - Professionals (EEO-1 Job Categories-United...
1,Hartwell,Andrew,7/24/2017,4/6/2018,0.70,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,"USA, MD, Rockville (One Preserve Pkwy)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Saifur Rahman (537235),NaN,14 Risk Assessment - Non Client Site,Technology,Software Engineering,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...
2,Vadicherla,Sudhakar,5/30/2017,4/6/2018,0.85,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Family Reasons,Terminate Employee > Voluntary > Benefits,NaN,...,"USA, VA, Herndon (575 Herndon Pkwy)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Christopher Cullerot (574970),NaN,14 Risk Assessment - Non Client Site,Technology,SD Data Engineer,Associate,2 - Professionals (EEO-1 Job Categories-United...
3,Barke,Jakob,5/31/2016,4/13/2018,1.87,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,"USA, MD, Lanham (5000 Ellin Rd)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,George Filippakis | Nick Filippakis (581389),NaN,23 Change Technology - Client Site,Technology,SD Project/Program Manager,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...
4,Zewdie,Kaleb,8/29/2016,4/13/2018,1.62,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Relocation,NaN,NaN,...,"USA, MD, Bethesda (9000 Rockville Pike)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Scot McGinn (520950),NaN,26 Op & Support Analysis - Client Site,Technology,Software Engineering,Associate,2 - Professionals (EEO-1 Job Categories-United...


In [4]:
df_hire['Hire Date'] = pd.to_datetime(df_hire['Hire Date'])
df_hire['Termination Date'] = pd.to_datetime(df_hire['Termination Date'])

In [5]:
df_hire.shape

(486, 29)

In [6]:
df_hire['Full Name']=df_hire['First Name'] + '' + df_hire['Last Name']

In [7]:
time_series_term=df_hire.groupby('Termination Date')['Full Name'].count()

In [8]:
time_series_hires=df_hire.groupby('Hire Date')['Full Name'].count()

In [9]:
time_series_term.isnull().sum()

0

In [10]:
time_series_hires.isnull().sum()

0

In [11]:
full_time_series_df=pd.merge(time_series_term,time_series_hires,how='outer',left_index=True, right_index=True)

In [12]:
full_time_series_df.fillna(0,inplace=True)

In [13]:
full_time_series_df.shape

(517, 2)

In [14]:
full_time_series_df.columns=['termination','hires']

In [15]:
full_time_series_df

,termination,hires
1998-09-28,0.0,1.0
1999-03-01,0.0,1.0
1999-03-15,0.0,1.0
2001-02-16,0.0,1.0
2002-01-24,0.0,1.0
2003-02-10,0.0,1.0
2003-10-20,0.0,1.0
2004-01-19,0.0,1.0
2004-03-08,0.0,1.0
2004-08-30,0.0,2.0


In [136]:
full_time_series_df.isnull().sum()

termination    0
hires          0
dtype: int64

In [65]:
from statsmodels.tsa.stattools import grangercausalitytests

test_result = grangercausalitytests(full_time_series_df[['Full Name_x', 'Full Name_y']], maxlag=7, verbose=False)
test_result

{1: ({'ssr_ftest': (6.316921130550219, 0.019413435997375744, 23.0, 1),
   'ssr_chi2test': (7.140867364969814, 0.007534670682386554, 1),
   'lrtest': (6.309436806022603, 0.012009698339406846, 1),
   'params_ftest': (6.316921130550225, 0.019413435997375706, 23.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (2.8912967352454118, 0.07889217833578023, 20.0, 2),
   'ssr_chi2test': (7.228241838113529, 0.026940597776760562, 2),
   'lrtest': (6.349182974845469, 0.04181118098992187, 2),
   'params_ftest': (2.8912967352454144, 0.07889217833578001, 20.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (2.1808623780492407, 0.12772357945990245, 17.0, 3),
   'ssr_chi2test': (9.236593601149726, 0.026305149533418757, 3),
   'lrtest': (7.814343731556811, 0.05000860958326077, 3),
   'params_ftest': (2.180862378049238, 0.1277235794599027, 17.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0.

In [66]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(full_time_series_df)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
Full Name_x ::  9.55      > 12.3212   =>   False
Full Name_y ::  3.7       > 4.1296    =>   False


In [91]:
nobs = 4
df_train, df_test = full_time_series_df[0:-nobs], full_time_series_df[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (4, 8)

(23, 2)
(4, 2)


In [92]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")    

In [93]:
# ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Full Name_x" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.7809
 No. Lags Chosen       = 0
 Critical value 1%     = -3.77
 Critical value 5%     = -3.005
 Critical value 10%    = -2.643
 => P-Value = 0.0001. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "Full Name_y" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -5.1569
 No. Lags Chosen       = 0
 Critical value 1%     = -3.77
 Critical value 5%     = -3.005
 Critical value 10%    = -2.643
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.




In [95]:
model = VAR(df_train)
for i in [1,2,3,4,5,6]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -1.0051596678907535
BIC :  -0.7076026351566673
FPE :  0.367239695839053
HQIC:  -0.9350641783116176 

Lag Order = 2
AIC :  -0.6923932274910303
BIC :  -0.1950015904798763
FPE :  0.5097889062260473
HQIC:  -0.5844465094339013 

Lag Order = 3
AIC :  -0.36828298697313655
BIC :  0.32872960451465705
FPE :  0.7360126522179097
HQIC:  -0.23221880646220838 

Lag Order = 4
AIC :  -0.4586119095943795
BIC :  0.4361197548790905
FPE :  0.745194577137761
HQIC:  -0.30718776313148677 

Lag Order = 5
AIC :  -0.24858350127473416
BIC :  0.8396486472650229
FPE :  1.1615361730588236
HQIC:  -0.09853096175952514 

Lag Order = 6
AIC :  0.10434711731678092
BIC :  1.3786734082262884
FPE :  2.9309588703299587
HQIC:  0.23101766368669807 



C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [97]:
x = model.select_order(maxlags=6)
x.summary()

,AIC,BIC,FPE,HQIC
0,-1.397*,-1.299*,0.2473*,-1.388*
1,-1.188,-0.8941,0.3071,-1.159
2,-0.9064,-0.4163,0.4187,-0.8577
3,-0.5552,0.1310,0.6368,-0.4870
4,-0.1920,0.6902,1.049,-0.1043
5,0.002107,1.080,1.640,0.1093
6,0.1043,1.379,2.931,0.2310


In [96]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 10, May, 2020
Time:                     21:45:34
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   0.436120
Nobs:                     19.0000    HQIC:                 -0.307188
Log likelihood:          -31.5629    FPE:                   0.745195
AIC:                    -0.458612    Det(Omega_mle):        0.343132
--------------------------------------------------------------------
Results for equation Full Name_x
                    coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------------
const                  0.170730         1.060755            0.161           0.872
L1.Full Name_x        -0.216355         0.236822           -0.914           0.361
L1.Full Name_y         0.298150         0.136637  

In [98]:
from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

for col, val in zip(full_time_series_df.columns, out):
    print(adjust(col), ':', round(val, 2))

NameError: name 'adjust' is not defined

In [100]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_train.values[-lag_order:]
forecast_input

4


array([[2, 1],
       [1, 1],
       [1, 1],
       [1, 1]], dtype=int64)

In [103]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_train.index[-nobs:], columns=df_train.columns + '_2d')
df_forecast

,Full Name_x_2d,Full Name_y_2d
2019-06-26,0.656832,2.118870
2019-07-10,1.177511,1.760146
2019-07-15,1.272285,1.779279
2019-07-18,1.342361,1.831618


In [106]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [110]:
df_results = invert_transformation(df_train, df_forecast, second_diff=True)        
df_results.loc[:, ['Full Name_x_2d', 'Full Name_y_2d']]

,Full Name_x_2d,Full Name_y_2d
2019-06-26,0.656832,2.118870
2019-07-10,1.177511,1.760146
2019-07-15,1.272285,1.779279
2019-07-18,1.342361,1.831618


In [112]:
df_test

,Full Name_x,Full Name_y
2019-07-31,1,1
2019-08-12,2,2
2019-12-09,1,1
2020-01-27,1,1


In [114]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: hire')
accuracy_prod = forecast_accuracy(df_results['Full Name_x_2d'].values, df_test['Full Name_x'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

print('\nForecast Accuracy of: term')
accuracy_prod = forecast_accuracy(df_results['Full Name_y_2d'].values, df_test['Full Name_y'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: hire
mape :  0.3423
me :  -0.1378
mae :  0.4451
mpe :  -0.0349
rmse :  0.4964
corr :  0.1399
minmax :  0.3059

Forecast Accuracy of: term
mape :  0.7124
me :  0.6225
mae :  0.7424
mpe :  0.6525
rmse :  0.8075
corr :  -0.4484
minmax :  0.385


In [14]:
#reading in this dataset just in case it's needed, will primarily be using the turnover set
df_turnover=pd.read_csv('../Data/turnover_data.csv')
df_turnover.head()

,Hire Date,Legal Name,Job Requisition,Create Job Requisition Reason,Hiring Manager,Recruiter,Principal,Job Profile,Management Level,Location,Job Requisition Eligibility Agency,Eligibility,Job Application Source,Source Category,Requisition Type
0,5/7/2018,"Bruce, Alexander H","R0025328 Digital Strategist, Mid (Filled)",Recruiting > New Requisition,Nishana Kuruppu (553726),NaN,Nancy Rhodes (541081),Digital IT Strategist Mid (inactive),Senior Consultant,"USA, DC, Washington (901 15th St NW)",None,None,Other,Booz Allen Hamilton websites,Sold and Funded
1,5/7/2018,"Raheem, Lynette J",R0022859 Privacy Engineer (Filled),Recruiting > New Requisition,Paul Simonoff (575378),NaN,NaN,Cyber Sec Pol & Comp Analyst Sr (inactive),Associate,"USA, DC, Washington (901 15th St NW)",DOD,None,Other,Job Board,Sold and Funded
2,5/21/2018,"Reed, Erin",R0027023 Energy Communications Specialist (Fil...,Recruiting > New Requisition,Lisa Brundage (580564),Shelby Lancaster (603308),Scott Thigpen (528431),Communications Spec Mid (inactive),Senior Consultant,"USA, VA, Arlington (1550 Crystal Dr Suite 300)...",DOD,Secret,Other,Job Board,Sold and Funded
3,5/29/2018,"Feather, Shellie",R0027563 Energy Performance Contracting Projec...,Recruiting > New Requisition,Christopher Watroba (544608),Shelby Lancaster (603308),NaN,Project Manager Sr (inactive),Associate,"USA, WA, Seattle (Home)",DOD,None,Other,Booz Allen Hamilton websites,Sold and Funded
4,6/4/2018,"Nayudu, Anupama",R0022502 Oracle Database Administrator (Filled),Recruiting > New Requisition,Oanh Dang (525109),Leroy Gifford | Roy Gifford (524804),James Hurst (534911),Database Admin Sr (inactive),Associate,"USA, VA, Herndon (575 Herndon Pkwy)",DOD,None,Referral,Referral,Sold and Funded


In [20]:
df_turnover.shape

(267, 15)

In [15]:
time_series_hires=df_turnover.groupby('Hire Date')['Legal Name'].count()